# Assignment 2 DD2358
 Group 17: Felix Schätzle, Georgios Mitsos, Thorben Fetz

## Exerice 1: Cythonize the STREAM Benchmark

## Task 1.1
Cython version of the STREAM benchmark with ctypes

In [2]:
from timeit import default_timer as timer
import numpy as np
import array
import matplotlib.pyplot as plt
from functools import wraps


In [3]:
STREAM_ARRAY_SIZE = 30_000_000

# LISTS
a = [1.0]*STREAM_ARRAY_SIZE
b = [2.0]*STREAM_ARRAY_SIZE
c = [0.0]*STREAM_ARRAY_SIZE

# ARRAYS
a_2 = array.array('f',[1.0]*STREAM_ARRAY_SIZE)
b_2 = array.array('f',[2.0]*STREAM_ARRAY_SIZE)
c_2 = array.array('f',[0.0]*STREAM_ARRAY_SIZE)

scalar = 2.0